In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set()

In [4]:
dataset = pd.read_csv('Churn_Modelling.csv', index_col = 'RowNumber')
dataset.head()


,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,,,,,,,,,,,,,
1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
#Customer ID and Surname would not be relevant as features
X_columns = dataset.columns.tolist()[2:12]
Y_columns = dataset.columns.tolist()[-1:]
print(X_columns)
print(Y_columns)

['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']
['Exited']


In [6]:
X = dataset[X_columns].values 
Y = dataset[Y_columns].values

In [7]:
#We need to encode categorical variables such as geography and gender
from sklearn.preprocessing import LabelEncoder
X_column_transformer = LabelEncoder()
X[:, 1] = X_column_transformer.fit_transform(X[:, 1])

In [8]:
#Lets Encode gender now
X[:, 2] = X_column_transformer.fit_transform(X[:, 2])

We are treating countries with ordinal values(0 < 1 < 2) but they are incomparable.
To solve this we can use one hot encoding.
We will perform some standardization 

In [9]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

pipeline = Pipeline(
    [
        ('Categorizer', ColumnTransformer(
            [
                ("Gender Label Encoder", OneHotEncoder(categories = 'auto', drop = 'first'), [2]),
                ("Geography Label Encoder", OneHotEncoder(categories = 'auto', drop = 'first'), [1])
            ], 
            remainder = 'passthrough', n_jobs = 1)),
        ('Normalizer', StandardScaler())
    ]
)

In [10]:
#Standardize the features
X = pipeline.fit_transform(X)

In [11]:
#Spilt the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [12]:
#Let us create the Neural Network
from keras.models import Sequential
from keras.layers import Dense, Dropout

2023-10-22 12:02:38.961912: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
#Initialize ANN
classifier = Sequential()

2023-10-22 12:02:42.911818: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-22 12:02:42.915498: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [14]:
#Add input layer and hidden layer
classifier.add(Dense(6, activation = 'relu', input_shape = (X_train.shape[1], )))
classifier.add(Dropout(rate = 0.1))

In [15]:
#Add second layer
classifier.add(Dense(6, activation = 'relu'))
classifier.add(Dropout(rate = 0.1))

In [16]:
#Add output layer
classifier.add(Dense(1, activation = 'sigmoid'))

In [17]:
#Let us take a look at our network
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 72        
                                                                 
 dropout (Dropout)           (None, 6)                 0         
                                                                 
 dense_1 (Dense)             (None, 6)                 42        
                                                                 
 dropout_1 (Dropout)         (None, 6)                 0         
                                                                 
 dense_2 (Dense)             (None, 1)                 7         
                                                                 
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [18]:
#Optimize the weights
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [19]:
#Fitting the Neural Network
history = classifier.fit(X_train, y_train, batch_size = 32, epochs = 20, validation_split = 0.1, verbose = 2)

Epoch 1/20
225/225 - 3s - loss: 0.6671 - accuracy: 0.6160 - val_loss: 0.5111 - val_accuracy: 0.7925 - 3s/epoch - 14ms/step
Epoch 2/20
225/225 - 1s - loss: 0.4997 - accuracy: 0.7908 - val_loss: 0.4616 - val_accuracy: 0.7950 - 1s/epoch - 5ms/step
Epoch 3/20
225/225 - 1s - loss: 0.4723 - accuracy: 0.7942 - val_loss: 0.4393 - val_accuracy: 0.7975 - 1s/epoch - 5ms/step
Epoch 4/20
225/225 - 1s - loss: 0.4608 - accuracy: 0.7979 - val_loss: 0.4228 - val_accuracy: 0.8000 - 1s/epoch - 5ms/step
Epoch 5/20
225/225 - 2s - loss: 0.4453 - accuracy: 0.8079 - val_loss: 0.4063 - val_accuracy: 0.8225 - 2s/epoch - 7ms/step
Epoch 6/20
225/225 - 1s - loss: 0.4333 - accuracy: 0.8186 - val_loss: 0.3935 - val_accuracy: 0.8400 - 1s/epoch - 5ms/step
Epoch 7/20
225/225 - 1s - loss: 0.4216 - accuracy: 0.8257 - val_loss: 0.3830 - val_accuracy: 0.8462 - 1s/epoch - 6ms/step
Epoch 8/20
225/225 - 1s - loss: 0.4105 - accuracy: 0.8272 - val_loss: 0.3716 - val_accuracy: 0.8500 - 1s/epoch - 5ms/step
Epoch 9/20
225/225 - 1s

In [20]:
y_pred = classifier.predict(X_test)
print(y_pred[:5])

63/63 [==============================] - 1s 6ms/step
[[0.2805454 ]
 [0.35112205]
 [0.13996741]
 [0.07166433]
 [0.1190502 ]]


In [21]:
#Let us use confusion matrix with cutoff value as 0.5
y_pred = (y_pred > 0.5).astype(int)
print(y_pred[:5])

[[0]
 [0]
 [0]
 [0]
 [0]]


In [22]:
#Making the Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1536   59]
 [ 233  172]]


In [23]:
#Accuracy of our NN
print(((cm[0][0] + cm[1][1])* 100) / len(y_test), '% of data was classified correctly')

85.4 % of data was classified correctly


In [24]:
!pip install ann_visualizer
!pip install graphviz

In [26]:
from ann_visualizer.visualize import ann_viz;

ann_viz(classifier, title="NN")